# Fine tune distilbert to perform Text classification 

This notebook is intended to train `text-classification` models based on `distilbert base uncased` model. To do so we are using [Transformers 🤗🤗](https://huggingface.co/docs/transformers/index).

### Considerations
- The dataset must have column "text" where all the input questions are setted
- An `S3 Instance` is required to correctly store the model

#### Install required libs   📥📥

In [1]:
!pip install transformers datasets evaluate accelerate  mlflow 


[notice] A new release of pip available: 22.2.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


## Dataset manipulation & env preparation

In [2]:
#Define your input column and your output column

input_column_name="text"
output_column_name="sentiment"

In [3]:
import pandas as pd
import os
labeled_dataset = "dataset.csv"

# Assuming the file is in the current working directory
df = pd.read_csv(labeled_dataset)

# Display the first few rows of the dataframe
df.head()

,text,sentiment
0,It is a charity school whose fees are calculat...,English
1,"John looked out the window of the show-house, ...",English
2,At least forty-one protesters were killed in t...,English
3,It was so small and undeveloped.,English
4,Do you mean it?,English


## Give a name to your model and version  🧙‍♂️🧙‍♂️

This process is crucial mainly because a `text-classification` model can be intended for a huge amount of approaches

In [4]:
import ipywidgets as widgets
from IPython.display import display

# Creating a box layout for better widget organization
box_layout = widgets.Layout(display='flex',
                            flex_flow='row',
                            justify_content='space-between',
                            width='50%')

# Text input widget
model_name_text = widgets.Text(
    description='Model Name:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(flex='2 1 auto', width='auto')
)

version_model_text = widgets.Text(
    description='Version:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(flex='2 1 auto', width='auto')
)

# Button widget
button_model = widgets.Button(description="Save",
                        button_style='primary',  # Apply a Bootstrap theme
                        layout=widgets.Layout(flex='1 1 auto', width='auto'))

button_version = widgets.Button(description="Save",
                        button_style='primary',  # Apply a Bootstrap theme
                        layout=widgets.Layout(flex='1 1 auto', width='auto'))

# Function to handle button click
def on_button_clicked(b):
    print("Saved text: ", text.value)

# Linking the button click event to the function
button_model.on_click(on_button_clicked)
button_version.on_click(on_button_clicked)

# Using a VBox for vertical stacking of widgets
widget_box_model = widgets.VBox([model_name_text, button_model], layout=box_layout)
widget_version = widgets.VBox([version_model_text, button_version], layout=box_layout)

# Displaying the widgets
display(widget_box_model)
display(widget_version)


In [5]:
model_name=model_name_text.value
version_model=version_model_text.value
print(f"Your model is going to be saved with name `{model_name}` & version `{version_model}`")
print(f"\n Under directory models/openvino/text-classification/{model_name}/{version_model}")
model_path_s3=f"models/openvino/text-classification/{model_name}/{version_model}"

Your model is going to be saved with name `language` & version `2`

 Under directory models/openvino/text-classification/language/2


### Model manipulation 

In [6]:
# Read json files label2id &  id2label
import json
import os
# Opening JSON file
file_label2id = open('label2id.json')
file_id2label = open('id2label.json')

label2id = json.load(file_label2id)
id2label=json.load(file_id2label)
print(f"The label2id json loaded correctly: {label2id}")
print(f"The id2label json loaded correctly: {id2label}")
os.remove("label2id.json")
os.remove("id2label.json")

The label2id json loaded correctly: {'Spanish': 0, 'Portuguese': 1, 'Italian': 2, 'English': 3, 'French': 4}
The id2label json loaded correctly: {'0': 'Spanish', '1': 'Portuguese', '2': 'Italian', '3': 'English', '4': 'French'}


In [7]:
# Add the new 'label' column to the dataframe by mapping values from the 'category' column
df['label'] = df[output_column_name].replace(label2id)

df.head(3)

,text,sentiment,label
0,It is a charity school whose fees are calculat...,English,3
1,"John looked out the window of the show-house, ...",English,3
2,At least forty-one protesters were killed in t...,English,3


In [8]:
test_size = 0.3 ## Define the testing size for metrics

In [9]:
from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(df, test_size=test_size)

In [10]:
from datasets import Dataset

train_dataset = Dataset.from_pandas(df_train)
test_dataset = Dataset.from_pandas(df_test)

In [11]:
from transformers import AutoTokenizer
base_model_name = 'distilbert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(base_model_name)

In [12]:
def preprocess_function(examples):
    return tokenizer(examples[input_column_name], truncation=True)

In [13]:
tokenized_train = train_dataset.map(preprocess_function, batched=True)
tokenized_test = test_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/2172 [00:00<?, ? examples/s]

Map:   0%|          | 0/931 [00:00<?, ? examples/s]

In [14]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(base_model_name, num_labels=len(label2id),id2label=id2label, label2id=label2id)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
from transformers import TrainingArguments, Trainer
import evaluate
import numpy as np
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [16]:
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
  logits, labels = eval_pred
  predictions = np.argmax(logits, axis=-1)
  return metric.compute(predictions=predictions, references=labels)

In [17]:
#Mlflow
import mlflow

os.environ["MFLOW_EXPERIMENT_NAME"]=model_name

## Training

### Run name in Mlflow
Your must set a run name in mlflow in order to identify this run.

In [18]:
# Creating a box layout for better widget organization
box_layout = widgets.Layout(display='flex',
                            flex_flow='row',
                            justify_content='space-between',
                            width='50%')

# Text input widget
run_name_text = widgets.Text(
    description='Run Name:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(flex='2 1 auto', width='auto')
)

# Button widget
button = widgets.Button(description="Save",
                        button_style='primary',  # Apply a Bootstrap theme
                        layout=widgets.Layout(flex='1 1 auto', width='auto'))

# Function to handle button click
def on_button_clicked(b):
    print("Saved text: ", text.value)

# Linking the button click event to the function
button.on_click(on_button_clicked)

# Using a VBox for vertical stacking of widgets
widget_box = widgets.VBox([run_name_text, button], layout=box_layout)

# Displaying the widgets
display(widget_box)

In [19]:
training_args = TrainingArguments(
    hub_model_id=model_name,
    output_dir=run_name_text.value,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    logging_strategy="epoch",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    eval_dataset=tokenized_test
)

/opt/app-root/lib64/python3.9/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [20]:
import mlflow

mlflow.set_tracking_uri(os.environ.get("MLFLOW_TRACKING_URI"))
mlflow.set_experiment(model_name)

<Experiment: artifact_location='mlflow-artifacts:/8', creation_time=1716408650285, experiment_id='8', last_update_time=1716410473987, lifecycle_stage='active', name='language', tags={}>

In [21]:
### Fine tune model
trainer.train()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Epoch,Training Loss,Validation Loss,Accuracy
1,0.514500,0.108085,0.974221
2,0.081500,0.083930,0.980666
3,0.041900,0.089652,0.978518
4,0.024900,0.089483,0.979592
5,0.017700,0.087286,0.977444


TrainOutput(global_step=680, training_loss=0.13606916911461775, metrics={'train_runtime': 1201.5319, 'train_samples_per_second': 9.038, 'train_steps_per_second': 0.566, 'total_flos': 92102886224640.0, 'train_loss': 0.13606916911461775, 'epoch': 5.0})

In [22]:
mlflow.end_run()

In [23]:
## Save pytorch 
trainer.save_model(model_name)

In [24]:
!pip install optimum[openvino,nncf] -q

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
codeflare-sdk 0.14.1 requires rich<13.0,>=12.5, but you have rich 13.7.1 which is incompatible.

[notice] A new release of pip available: 22.2.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [25]:
from optimum.intel import OVModelForSequenceClassification

ov_model= OVModelForSequenceClassification.from_pretrained(model_name,export=True)
ov_model.save_pretrained(f"{model_name}_openvino")

/opt/app-root/lib64/python3.9/site-packages/networkx/utils/backends.py:135: RuntimeWarning: networkx backend defined more than once: nx-loopback
  backends.update(_get_backends("networkx.backends"))


INFO:nncf:NNCF initialized successfully. Supported frameworks detected: torch, onnx, openvino


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
No CUDA runtime is found, us

In [26]:
## Load the bucket and file path
bucket_name =  "rowdy"

In [27]:
## AWS_ACCESS_KEY_ID & AWS_SECRET_ACCESS_KEY should be set as Env variables
key_id = os.environ.get('S3_ACCESS_KEY_ID')
secret_key = os.environ.get('S3_SECRET_ACCESS_KEY')

## Upload all remaining files to s3 and delete them

In [28]:
import boto3
from datetime import datetime


session = boto3.session.Session()
s3_client = session.client('s3',
                        region_name='nyc3',
                        endpoint_url='https://nyc3.digitaloceanspaces.com',
                        aws_access_key_id=key_id,
                        aws_secret_access_key=secret_key)

def uploadModel(path):
     for root,dirs,files in os.walk(path):
            for file in files:
                s3_client.upload_file(os.path.join(root,file),bucket_name,f"models/openvino/text-classification/{model_name}/{version_model}/{file}")

In [29]:
#Upload ov binary to S3 & mlflow run
uploadModel(f"{model_name}_openvino")

In [30]:
## Delete directories in Jupyter Notebook
import shutil

# Remove the local model directory
shutil.rmtree(model_name)
os.remove(labeled_dataset)
shutil.rmtree(f"{model_name}_openvino")